In [ ]:
from diffusers import ControlNetModel, AutoencoderKL, EulerAncestralDiscreteScheduler
from pipeline_test import StableDiffusionXLInpaintControlNetPipeline
from diffusers.utils import load_image
from PIL import Image
import numpy as np
import torch
import cv2

In [ ]:
controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-sdxl-1.0",
    torch_dtype=torch.float16
)
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = StableDiffusionXLInpaintControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    torch_dtype=torch.float16,
)
pipe.enable_model_cpu_offload()

In [ ]:
prompt = "aerial view, a futuristic research complex in a bright foggy jungle, hard lighting"
negative_prompt = 'low quality, bad quality, sketches'

image = load_image("/home/viper/Result/Week25/input.jpg")
mask = load_image("/home/viper/Result/Week25/mask.jpg")

controlnet_conditioning_scale = 0.5  # recommended for good generalization

c_image = np.array(mask)
c_image = cv2.Canny(c_image, 100, 200)
c_image = c_image[:, :, None]
c_image = np.concatenate([c_image, c_image, c_image], axis=2)
c_image = Image.fromarray(c_image)

generator = torch.Generator(device="cuda").manual_seed(1145141919)

pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
images = pipe(
                prompt, 
                negative_prompt=negative_prompt,
                image=image, 
                mask_image=mask, 
                control_image=c_image,
                num_inference_steps=50,
                generator=generator
    ).images

images[0].save(f"/home/viper/Result/Week26/ci-final.png")